In [1]:
import requests
from google.colab import files
import shutil
from bs4 import BeautifulSoup

In [2]:
import html
print(html.unescape('&pound;682m'))

£682m


In [3]:
%rm -rf 1-100
%rm -rf 1-100_content
%rm -rf 1-100_content-text
%rm -rf 1-100_soup-text
%rm -rf logs

!mkdir 1-100
!mkdir 1-100_content
!mkdir 1-100_content-text
!mkdir 1-100_soup-text
!mkdir logs

In [4]:
#################### TIME ####################

import time;
def fill_time(arg_time):
	if len(arg_time) < 2:
		res = "0" + arg_time
	else:
		res = arg_time
	return res

def absvalue(arg_time):
  if arg_time < 0:
    return (24 + arg_time)
  else:
    return arg_time

def get_timestamp(arg_tz):
  localtime = time.localtime(time.time())
  time_str = time.asctime(time.localtime(time.time()))
  y = fill_time(str(localtime[0])[2:4])
  mo = fill_time(str(localtime[1]))
  d = fill_time(str(localtime[2]))
  h = fill_time(str(absvalue(localtime[3] + arg_tz)))
  min = fill_time(str(localtime[4]))
  s = fill_time(str(localtime[5]))
  timestamp = y + mo + d + "-" + h + min + s
  # print("Local current time :", time_str, "timestamp:", timestamp)
  return timestamp

In [5]:
import re

def remove_tags(arg_line_str):
  res_str = re.sub(r"<[^>]*>", "", arg_line_str)
  return(res_str.strip())

def remove_3_newlines(arg_line_str):
  res_str = re.sub(r"(\\n){3,}", "\n\n", arg_line_str)
  return(res_str)

In [6]:
def write_log(arg_text, arg_log_file):
  fname = "/content/logs/" + arg_log_file + ".txt"
  f = open(fname, "a")
  f.write(arg_text)
  f.close()

In [7]:
def write_file(arg_directory, arg_fname, arg_ext, arg_text, arg_log_file, arg_mode = "w"):
  fname = arg_directory + "/" + arg_fname + arg_ext
  f = open(fname, arg_mode)
  f.write(arg_text)
  f.close()
  log_txt = "Written \"" + arg_fname + " \"as \"" + arg_ext + "\" to \"" + arg_directory + "\" containing " + str(len(arg_text)) + " characters in " + str(len(arg_text.split("\n"))) + " lines."
  # print(log_txt)
  write_log(log_txt + "\n", arg_log_file)

In [8]:
def make_scp_idx(arg_scp):
  if len(str(arg_scp)) < 3:
    scp_str = ("0" * (3 - len(str(arg_scp)))) + str(arg_scp)
    return(scp_str)
  return(str(arg_scp))

# <div class="page-tags">
#   <span>
#       <a href="/system:page-tags/tag/_cc#pages">_cc</a><a href="/system:page-tags/tag/_licensebox#pages">_licensebox</a><a href="/system:page-tags/tag/euclid#pages">euclid</a><a href="/system:page-tags/tag/liquid#pages">liquid</a><a href="/system:page-tags/tag/scp#pages">scp</a><a href="/system:page-tags/tag/self-replicating#pages">self-replicating</a><a href="/system:page-tags/tag/thermal#pages">thermal</a>
#   </span>
# </div>

tags_to_ignore = ["_cc", "_licensebox"]
# get page tags
def get_page_tags(arg_scp_str, arg_text, arg_dict):
  tags_html = arg_text.split("<div class=\"page-tags\">")[1].split("</div>")[0].strip()
  soup_tags = BeautifulSoup(tags_html, 'html.parser').get_text(separator=u' ').strip().split(" ")
  tags = [tag for tag in soup_tags if tag not in tags_to_ignore]
  # print(tags)
  arg_dict[arg_scp_str] = tags

  return (", ".join(tags))

# get the html file
def write_raw_html_file(arg_directory, arg_scp_str, arg_text, arg_log_file):
  write_file(arg_directory, "scp-" + arg_scp_str, ".html", arg_text, arg_log_file)

# between main content and scripts
def write_html_section_file(arg_directory, arg_scp_str, arg_text, arg_log_file):
  content = arg_text.split("<div id=\"main-content\">")[1].split("<script>")[0].strip()
  soup_content = BeautifulSoup(content, 'html.parser').prettify()
  write_file(arg_directory, "scp-" + arg_scp_str, ".html", soup_content, arg_log_file)
  return(content)

# no tags
def write_text_content_file(arg_directory, arg_scp_str, arg_text, arg_log_file):
  text_content = html.unescape(remove_tags(arg_text).strip())
  write_file(arg_directory, "scp-" + arg_scp_str, ".txt", text_content, arg_log_file)
  # return(text_content)

# append line
def append_to_p_tag(arg_line):
  try:
    if "<p" in arg_line.strip()[:2]:
      return("\n<p>\n" + arg_line.strip())
    else:
      return(arg_line.strip())
  except:
    return(arg_line.strip())

# souped text 
def write_soup_text_file(arg_directory, arg_scp_str, arg_text, arg_log_file):
  text_lines = str(BeautifulSoup(arg_text, 'html.parser')).split("\n")
  reconstructed_text = ""
  for line in text_lines:
    reconstructed_text += append_to_p_tag(line) + "\n"
  reconstructed_text = remove_3_newlines(BeautifulSoup(reconstructed_text, 'html.parser').get_text().strip())
  write_file(arg_directory, "scp-" + arg_scp_str, ".txt", reconstructed_text, arg_log_file)

def get_page_source_text(arg_scp, arg_dict, arg_log_file):
  scp_str = make_scp_idx(arg_scp)
  URL = "https://scp-wiki.wikidot.com/scp-" + scp_str
  page = requests.get(URL)
  write_raw_html_file("/content/1-100", scp_str, page.text, arg_log_file)
  tags = get_page_tags(scp_str, page.text, arg_dict)
  try:
    html_main_content = write_html_section_file("/content/1-100_content", scp_str, page.text, arg_log_file)
    write_soup_text_file("/content/1-100_soup-text", scp_str, html_main_content, arg_log_file)
    write_text_content_file("/content/1-100_content-text", scp_str, html_main_content, arg_log_file)
  except:
    print("Error with SCP-" + str(arg_scp))
    write_log("Error with SCP-" + str(arg_scp) + ".\n", arg_log_file)
  return(page.text)

In [9]:
# log_fname = "log_" + get_timestamp(-5)
# scp_str = make_scp_idx(29)
# URL = "https://scp-wiki.wikidot.com/scp-" + scp_str
# page = requests.get(URL)
# html_main_content = write_html_section_file("/content/1-100_content", scp_str, page.text, log_fname)
# write_soup_text_file("/content/1-100_soup-text", scp_str, html_main_content, log_fname)

In [10]:
tags_dict = {}
log_fname = "log_" + get_timestamp(-5)
for i in range(0, 101):
  x = get_page_source_text(i, tags_dict, log_fname)
print(tags_dict)

['pattern-screamer', 'researcher-rosen', 'tale']
['hub']
['alive', 'euclid', 'featured', 'scp', 'structure', 'transfiguration']
['alive', 'biological', 'computer', 'director-moose', 'euclid', 'featured', 'k-class-scenario', 'mind-affecting', 'organic', 'radioactive', 'rewrite', 'scp', 'self-replicating', 'telepathic', 'thermal', 'transfiguration']
['euclid', 'key', 'mind-affecting', 'portal', 'scp', 'spacetime', 'structure', 'temporal']
['adaptive', 'key', 'metallic', 'safe', 'scp', 'sentient']
['liquid', 'location', 'medical', 'rewrite', 'safe', 'scp', 'self-repairing']
['alive', 'euclid', 'humanoid', 'miniature', 'planet', 'scp']
['alive', 'biohazard', 'contagion', 'euclid', 'reanimation', 'scp', 'transfiguration']
['euclid', 'liquid', 'scp', 'self-replicating', 'thermal']
['artifact', 'compulsion', 'electronic', 'mind-affecting', 'safe', 'scp']
['autonomous', 'humanoid', 'military', 'safe', 'sapient', 'scp', 'sentient', 'statue']
['cognitohazard', 'document', 'euclid', 'mind-affecti